In [21]:
look_back = 6
batch_size = 1
loss = 'mean_squared_error'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import tensorflow as tf

np.random.seed(42)

df = pd.read_excel('../data/state_month_overdose.xlsx')

df['Deaths'] = df['Deaths'].apply(lambda x: 0 if x == 'Suppressed' else int(x))

df['Month'] = pd.to_datetime(df['Month'])
df.set_index('Month', inplace=True)

df = df.reset_index() #2015/01
df['Month Code'] = pd.to_datetime(df['Month Code'])#.reset_index() #2015-01-01
# df.set_index('Month', inplace=True)
df = df.groupby(['Month']).agg({'Deaths': 'sum'}).reset_index()



train = df[df['Month'] < '2019-01-01']
test = df[(df['Month'] >= '2019-01-01') & (df['Month'] <= '2019-12-01')]
testog = test
test = test.reset_index().drop(columns = ['index'])

2025-01-19 00:08:10.753343: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-19 00:08:10.776247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-19 00:08:10.816384: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-19 00:08:10.823190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 00:08:10.838300: I tensorflow/core/platform/cpu_feature_guar

In [2]:
test

,Month,Deaths
0,2019-01-01,3931
1,2019-02-01,3727
2,2019-03-01,4285
3,2019-04-01,3934
4,2019-05-01,4100
5,2019-06-01,4112
6,2019-07-01,4296
7,2019-08-01,4371
8,2019-09-01,4252
9,2019-10-01,4529


In [3]:
# Modify the create_dataset function to use a lookback of 3 months
def create_dataset(dataset, look_back=3):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset.iloc[i:(i + look_back)]  # Collect the previous 'look_back' months
        dataX.append(a)
        dataY.append(dataset.iloc[i + look_back])  # The target is the subsequent month
    return np.array(dataX), np.array(dataY)

look_back = 3

extended_test = pd.concat([train.iloc[-look_back:], test])

# Prepare LSTM datasets
trainX, trainY = create_dataset(train['Deaths'], look_back)
testX, testY = create_dataset(extended_test['Deaths'], look_back)

In [4]:
# Reshape inputs to match LSTM input requirements (samples, time_steps, features)
trainX = trainX.reshape((trainX.shape[0], look_back, 1))
testX = testX.reshape((testX.shape[0], look_back, 1))

In [5]:
# Rebuild the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1)

# model = Sequential([
#     LSTM(50, return_sequences=False, input_shape=(look_back, 1)),
#     Dense(1)
# ])
# model.compile(optimizer='adam', loss='mse')
# model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1)

I0000 00:00:1737274093.260769   14570 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-19 00:08:13.349551: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/sukikrishna/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 15870202.0000
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9216045.0000
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - loss: 2795126.2500
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - loss: 67137.8359
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step - loss: 29304.8359
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - loss: 33428.1641
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 64718.8281  
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 40285.0742 
Epoch 9/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 63550.0039
Epoch 10/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 46482.4492
Epoch 11/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 44858.6602 
Epoch 12/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - loss: 49832.2578
Epoch 13/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step - loss: 46472.9844
Epoch 14/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - loss: 4

In [6]:
print('Finished Running')

Finished Running


In [7]:
# Updated generate_forecast function to handle 3-month lookback
def generate_forecast(model, initial_sequence, num_predictions=12, look_back=3):
    predictions = []
    for _ in range(num_predictions):
        # Generate the next prediction
        next_prediction = model.predict(initial_sequence)
        predictions.append(next_prediction[0][0])
        
        # Update the sequence with the new prediction
        new_sequence = np.append(initial_sequence[0, 1:], [[next_prediction[0][0]]], axis=0)
        initial_sequence = new_sequence.reshape((1, look_back, 1))

    return np.array(predictions)

In [8]:
# Prepare the initial sequence for forecasting using the last `look_back` months from training
initial_sequence = trainX[-1].reshape((1, look_back, 1))
initial_sequence

array([[[3882],
        [3999],
        [3692]]])

In [9]:
# Generate test predictions with the updated lookback logic
testPredict = generate_forecast(model, initial_sequence, num_predictions=testY.shape[0], look_back=look_back)
trainPredict = model.predict(trainX)

# Flatten predictions for visualization and evaluation
testPredictlst = testPredict.flatten().tolist()
trainPredictlst = trainPredict.flatten().tolist()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [10]:
len(testPredictlst)

12

In [11]:
len(trainPredictlst)

45

In [12]:
combined_array = [0] * look_back + trainPredictlst + testPredictlst

In [13]:
val_df = df[df['Month'] <= '2019-12-01']
val_df

,Month,Deaths
0,2015-01-01,2771
1,2015-02-01,2627
2,2015-03-01,2907
3,2015-04-01,2822
4,2015-05-01,2850
5,2015-06-01,2634
6,2015-07-01,2734
7,2015-08-01,2810
8,2015-09-01,2925
9,2015-10-01,2970


In [14]:
len(val_df)

60

In [15]:
len(combined_array)

60

In [16]:
testX

array([[[3999],
        [3692],
        [3976]],

       [[3692],
        [3976],
        [3931]],

       [[3976],
        [3931],
        [3727]],

       [[3931],
        [3727],
        [4285]],

       [[3727],
        [4285],
        [3934]],

       [[4285],
        [3934],
        [4100]],

       [[3934],
        [4100],
        [4112]],

       [[4100],
        [4112],
        [4296]],

       [[4112],
        [4296],
        [4371]],

       [[4296],
        [4371],
        [4252]],

       [[4371],
        [4252],
        [4529]],

       [[4252],
        [4529],
        [4560]]])

In [17]:
sarima_model = SARIMAX(train['Deaths'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12),
                enforce_stationarity=False,
                enforce_invertibility=False)
sarima_result = sarima_model.fit(disp=False)
sarima_predictions = sarima_result.predict(start=0, end=len(train) + len(test) - 1, dynamic=False)

/home/sukikrishna/.local/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


In [18]:
len(sarima_predictions)

60

In [19]:
val_df['LSTM Predictions'] = combined_array
val_df['SARIMA Predictions'] = sarima_predictions

/tmp/ipykernel_14570/3914168234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['LSTM Predictions'] = combined_array
/tmp/ipykernel_14570/3914168234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['SARIMA Predictions'] = sarima_predictions


In [20]:
val_df.to_csv(f'../tables/{look_back}month_predictionresults_batch_{batch_size}_loss_{loss}.csv')